In [1]:
from kernel_submission import get_clean_data
data = get_clean_data()

/home/minnie/.pyenv/versions/3.6.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/minnie/.pyenv/versions/3.6.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [22]:
import pandas as pd
from kernel_submission import get_balanced_data, household_id
valid_df = get_balanced_data(data, 100)
train_df = pd.concat([data.reset_index(), valid_df.reset_index()]).drop_duplicates(keep=False).set_index(household_id)

In [24]:
from data_cleaning import target_table_breakdown
from kernel_submission import convert_to_binary_targets, run_train, target_column
from sklearn.metrics import classification_report

def run_test(clf, k_features, valid_data, target_value):    
    is_n_valid = valid_data.loc[valid_data[target_column]<=target_value]
    is_n_valid = convert_to_binary_targets(is_n_valid, target_value)
    sample_max = target_table_breakdown(is_n_valid)['total'].max()
    is_n_valid = get_balanced_data(is_n_valid, sample_max, random_state=10)
    preds = clf.predict(is_n_valid[k_features])
    print(preds)
#     print(classification_report(is_n_valid[target_column], preds))
    return list(zip(is_n_valid.index, preds))
    
def train_and_test(train_data, valid_data, clf):
    results = {}
    for k,v in clf.items():
        knn, k_features = run_train(v, train_data, k)
        results[k] = run_test(v, k_features, valid_data, k)
    return results

### Linear SVC

In [17]:
from sklearn.svm import LinearSVC
from kernel_submission import train_all_clf

clfs = {4:LinearSVC(C=100), 
        3:LinearSVC(C=100), 
        2:LinearSVC(C=100)}

results = train_and_test(train_df, valid_df, clfs)

             precision    recall  f1-score   support

          0       1.00      0.08      0.14       300
          1       0.27      1.00      0.42       100

avg / total       0.82      0.31      0.21       400

             precision    recall  f1-score   support

          0       0.66      0.68      0.67       200
          1       0.32      0.31      0.32       100

avg / total       0.55      0.55      0.55       300

             precision    recall  f1-score   support

          0       0.25      0.02      0.04       100
          1       0.49      0.94      0.64       100

avg / total       0.37      0.48      0.34       200



In [18]:
from kernel_submission import person_id

results_dfs = []
for k,v in sorted(results.items()):
    results_dfs.append(pd.DataFrame(v, columns=[person_id,'clf_%d'%k]).set_index(person_id))

results = pd.concat(results_dfs, axis=1, sort=False).fillna(0).astype('int')
results['clf_1'] = (~results[['clf_2','clf_3','clf_4']].any(axis=1)).astype('int')

In [19]:
from kernel_submission import compress_columns
targets = compress_columns(results, 'Preds', ['clf_1','clf_2','clf_3','clf_4'])+1

In [20]:
clf_report = targets.join(valid_df[[target_column]])
print(classification_report(clf_report[target_column], clf_report['Preds']))

             precision    recall  f1-score   support

          1       0.00      0.00      0.00       100
          2       0.49      0.94      0.64       100
          3       0.94      0.31      0.47       100
          4       0.59      1.00      0.74       100

avg / total       0.50      0.56      0.46       400



In [12]:
clfs = {4:LinearSVC(C=100), 
        3:LinearSVC(C=100), 
        2:LinearSVC(C=100)}
print(classification_report(clf_report[target_column], clf_report['Preds']))

             precision    recall  f1-score   support

          1       0.04      0.01      0.02       100
          2       0.51      1.00      0.67       100
          3       0.98      0.85      0.91       100
          4       0.95      0.88      0.91       100

avg / total       0.62      0.69      0.63       400



### Random Forest

In [25]:
from sklearn.ensemble import RandomForestRegressor

clfs = {4:RandomForestRegressor(n_estimators = 10, random_state = 42), 
        3:RandomForestRegressor(n_estimators = 10, random_state = 42), 
        2:RandomForestRegressor(n_estimators = 10, random_state = 42)}

results = train_and_test(train_df, valid_df, clfs)

[0.3        1.         1.         0.5        0.6        0.1
 0.4        0.8        1.         0.2        1.         0.2
 0.9        0.3        0.9        0.93333333 0.7        1.
 1.         0.2        1.         0.9        0.3        0.4
 0.4        0.3        0.2        0.4        1.         0.7
 0.475      0.64603175 0.3        0.4        0.6        0.3
 0.35       0.4        0.2        0.5        0.3        0.3
 0.1        0.4        0.4        0.2        0.4        0.405
 0.9        0.8        1.         0.3        0.2        0.6
 0.5        1.         0.2        1.         0.9        0.4
 1.         0.3        0.4        0.8        0.4        0.7
 0.4        0.8        0.92       1.         0.5        0.
 0.48333333 1.         1.         0.9        0.1        0.3
 0.6        0.4        0.2        1.         0.6        1.
 0.4        0.7        0.4        0.6        0.6        0.9
 0.         1.         0.2        1.         0.7        0.1
 1.         0.3        1.         0.4    